In [ ]:
# import fitz
# import pandas as pd
# import re

# # Abrir el PDF y extraer el texto completo
# doc = fitz.open("File.pdf")
# texto = ""
# for pagina in doc:
#     texto += pagina.get_text()
# # Dividir el texto por bloques que empiezan con una fecha de operación
# bloques = re.split(r"(?=\d{1,2} \w{3} 20\d{2}\nF\. valor:)", texto.strip())
# # Procesar cada bloque
# datos = []
# for  bloque in bloques:
#     lineas = bloque.strip().split('\n')
#     if len(lineas) <3:
#         continue #--> por qué
#     fecha_operacion = lineas[0].strip()
#     operacion = " ".join(lineas[2: -1]).strip()
#     importe_saldo = lineas[-1].strip()
# # Buscar importe y saldo con expresión regular
#     match = re.match(r"([−-]?\d+,\d{2})€\s+(\d+,\d{2})€", importe_saldo)
#     if match: 
#         importe, saldo = match.groups()
#         importe = float(importe.replace(',', '.').replace('−', '-'))  # convertir a float y manejar signo unicode
#         saldo = float(saldo.replace(',', '.'))
#         datos.append([fecha_operacion, operacion, importe, saldo])

# # Crear un DataFrame
# df = pd.DataFrame(datos, columns=["Fecha", "Descripción", "Importe (€)", "Saldo (€)"])

# # Mostrar las primeras filas
# print(df.head())
# print(df)


Empty DataFrame
Columns: [Fecha, Descripción, Importe (€), Saldo (€)]
Index: []
Empty DataFrame
Columns: [Fecha, Descripción, Importe (€), Saldo (€)]
Index: []


In [ ]:
# import fitz  # PyMuPDF
# import pandas as pd
# import re

# # Cargar el PDF y extraer el texto completo
# doc = fitz.open("File.pdf")
# texto = ""
# for pagina in doc:
#     texto += pagina.get_text()

# # Expresión regular para capturar fecha, operación, importe y saldo
# patron = re.compile(
#     r"(?P<fecha>\d{1,2} \w{3} 20\d{2})\s+F\. valor:.*?\n(?P<operacion>.+?)(?P<importe>[−\-]?\d{1,3}(?:\.\d{3})*,\d{2})€\s+(?P<saldo>\d{1,3}(?:\.\d{3})*,\d{2})€",
#     re.DOTALL
# )

# # Buscar coincidencias
# matches = patron.finditer(texto)

# # Procesar los datos
# datos = []
# for match in matches:
#     fecha = match.group("fecha").strip()
#     operacion = match.group("operacion").replace('\n', ' ').strip()
#     importe = float(match.group("importe").replace('.', '').replace(',', '.').replace('−', '-'))
#     saldo = float(match.group("saldo").replace('.', '').replace(',', '.'))
#     datos.append([fecha, operacion, importe, saldo])

# # Crear el DataFrame
# df = pd.DataFrame(datos, columns=["Fecha", "Descripción", "Importe (€)", "Saldo (€)"])

# # Mostrar una vista previa
# print(df.head())

# # Guardar si lo deseas
# # df.to_csv("movimientos_bancarios.csv", index=False)


         Fecha                                        Descripción  \
0  22 abr 2025  Transferencia Inmediata De Evelymar Bastidas R...   
1  22 abr 2025  Transferencia Inmediata De Evelymar Bastidas R...   
2  22 abr 2025  Transferencia Inmediata De Evelymar Bastidas R...   
3  21 abr 2025  Pago Movil En Heladeria Quint, Jinamar, Tarj. ...   
4  21 abr 2025  Bizum A Favor De Ruth Santana Gonzalez Concept...   

   Importe (€)  Saldo (€)  
0         15.0     840.16  
1         75.0     825.16  
2        300.0     750.16  
3        -10.6     450.16  
4        -10.0     460.76  


In [5]:
import fitz  # PyMuPDF
import pandas as pd
import re

# Ruta al archivo PDF
pdf_path = "File.pdf"  # Asegúrate de que este archivo esté en el mismo directorio

# Abrir el PDF
doc = fitz.open(pdf_path)

# Extraer texto completo
texto = ""
for pagina in doc:
    texto += pagina.get_text()

# Patrón para extraer datos: fecha, operación, importe y saldo
patron = re.compile(
    r"(?P<fecha>\d{1,2} \w{3} 20\d{2})\s+F\. valor:.*?\n(?P<operacion>.*?)(?P<importe>[−\-]?\d{1,3}(?:\.\d{3})*,\d{2})€\s+(?P<saldo>\d{1,3}(?:\.\d{3})*,\d{2})€",
    re.DOTALL
)

# Buscar movimientos
matches = patron.finditer(texto)

# Procesar resultados
datos = []
for match in matches:
    fecha = match.group("fecha").strip()
    operacion = match.group("operacion").replace('\n', ' ').strip()
    importe_str = match.group("importe").replace('−', '-').replace('.', '').replace(',', '.')
    saldo_str = match.group("saldo").replace('.', '').replace(',', '.')
    try:
        importe = float(importe_str)
        saldo = float(saldo_str)
        datos.append([fecha, operacion, importe, saldo])
    except ValueError:
        continue

# Crear DataFrame
df_pdf1 = pd.DataFrame(datos, columns=["Fecha_operacion", "Operacion", "Importe", "Saldo"])

# Guardar como CSV
df_pdf1.to_csv("movimientos_bancarios.csv", index=False, encoding="utf-8-sig")

print(f"✔ Dataset generado correctamente con {len(df_pdf1)} movimientos.")


✔ Dataset generado correctamente con 1535 movimientos.


In [3]:
import fitz  # PyMuPDF
import pandas as pd
import re

# 1. Ruta al archivo PDF (ajusta el nombre si es distinto)
pdf_path = "movimientosgenerados_07052025.pdf"

# 2. Abrir y leer todo el texto del PDF
doc = fitz.open(pdf_path)
texto = ""
for pagina in doc:
    texto += pagina.get_text()

# 3. Separar el texto por líneas
lineas = texto.split('\n')
movimientos = []

# 4. Analizar línea por línea para extraer movimientos
i = 0
while i < len(lineas):
    linea = lineas[i].strip()
    
    # Usamos regex para detectar líneas de movimientos
    match = re.match(
        r"^(\d{2}-\d{2}-\d{2})\s+(?:\d{2}-\d{2}-\d{2}\s+){1,2}\d+\s+(.*?)(\d{1,3}(?:,\d{2}))\s+([DH])(?:\s+(\d{1,3}(?:,\d{2}))\s+H)?", 
        linea
    )
    
    if match:
        # Extraer los valores
        fecha = match.group(1)
        descripcion = match.group(2).strip()
        importe = float(match.group(3).replace(',', '.'))
        signo = -1 if match.group(4) == 'D' else 1
        importe *= signo
        saldo = match.group(5)
        saldo = float(saldo.replace(',', '.')) if saldo else None

        # 5. Ver si la línea siguiente es parte del concepto (no empieza por fecha)
        concepto_extra = lineas[i + 1].strip() if i + 1 < len(lineas) else ""
        if concepto_extra and not re.match(r"^\d{2}-\d{2}-\d{2}", concepto_extra):
            descripcion += " " + concepto_extra
            i += 1  # Saltamos esa línea extra

        # Guardar el movimiento
        movimientos.append([fecha, descripcion, importe, saldo])
    
    i += 1

# 6. Crear DataFrame y limpiar fechas
df_pdf2 = pd.DataFrame(movimientos, columns=["Fecha_operacion", "Operacion", "Importe", "Saldo"])
df_pdf2["Fecha_operacion"] = pd.to_datetime(df["Fecha_operacion"], format="%d-%m-%y")

# 7. Guardar como CSV
df_pdf2.to_csv("movimientos_bancarios_extraidos.csv", index=False, encoding="utf-8-sig")

print(f"✔ Dataset guardado correctamente con {len(df_pdf2)} movimientos.")




✔ Dataset guardado correctamente con 826 movimientos.
